In [1]:
import pandas as pd
import numpy as np

In [2]:
#load datasets
train = pd.DataFrame(pd.read_csv('../datasets/train.csv'))
test = pd.DataFrame(pd.read_csv('../datasets/test.csv'))
spray = pd.DataFrame(pd.read_csv('../datasets/spray.csv'))
weather = pd.DataFrame(pd.read_csv('../datasets/weather.csv'))

In [13]:
#8. create weather_new df with index as Station
weather_new = weather.set_index('Station')

In [14]:
#9. apply average logic from step 7 to weather_new df
    #for both cols with M values, imputing 0.0
        #mostly affects col Water1
newcols = {}

for col in weather_new.drop(['Date', 'CodeSum'], axis = 1):
    newvals = []
    
    for date in weather_new['Date'].unique():
        val1 = weather_new[weather_new['Date'] == date].loc[1, col]
        val2 = weather_new[weather_new['Date'] == date].loc[2, col]
        
        if val1 not in ['M', '-', ' ', '  T'] and val2 not in ['M', '-', ' ', '  T']:
            aveval = np.average([pd.to_numeric(val1), pd.to_numeric(val2)])
            newvals.append(aveval)
            #print (col, date, aveval)
        elif val1 not in ['M', '-', ' ', '  T'] and val2 in ['M', '-', ' ', '  T']:
            newvals.append(val1)
            #print (col, date, val1)
        elif val1 in ['M', '-', ' ', '  T'] and val2 not in ['M', '-', ' ', '  T']:
            newvals.append(val2)
            #print (col, date, val2)
        else:
            newvals.append(0.0)
            #print ('Problem')
            
    newcols[col] = newvals
    #print (newvals)
print (newcols.keys())

dict_keys(['Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint', 'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset', 'Depth', 'Water1', 'SnowFall', 'PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed'])


In [15]:
#10. create codesum df for handling CodeSum text abbreviations (denotes weather phenomena like thunderstorm, snow, smoke, etc)
codesum1 = weather[weather['Station'] == 1]['CodeSum'].to_frame()
codesum1 = codesum1.set_index(weather[weather['Station'] == 1]['Date'])
codesum1.index.rename('Date')

codesum2 = weather[weather['Station'] == 2]['CodeSum'].to_frame()
codesum2 = codesum2.set_index(weather[weather['Station'] == 2]['Date'])
codesum2.index.rename('Date')

codesum = codesum1.join(codesum2, lsuffix = '1', rsuffix = '2')
print(codesum.shape)

(1472, 2)


In [16]:
#11. check if CodeSum from each Station matches
    #a. if match: use CodeSum
    #b. if no_match: use CodeSum with more information (non-empty Codesum, aggregate Codesum)

exact_matches = 0
both_null = 0
partial_matches = 0
stat1_null = 0
stat2_null = 0

aggregate = []

for date in codesum.index:
    #print(date)
    
    if codesum['CodeSum1'][date] == ' ' and codesum['CodeSum2'][date] == ' ':
        both_null += 1
        aggregate.append(' ')
    elif codesum['CodeSum1'][date] == codesum['CodeSum2'][date]:
        exact_matches += 1
        aggregate.append(codesum['CodeSum1'][date])
        #print(matches)
    else:
        
        if len(codesum['CodeSum1'][date]) == 1 and len(codesum['CodeSum2'][date]) > 1:
            stat1_null +=1
            aggregate.append(codesum['CodeSum2'][date])
            #print(date, codesum['CodeSum1'][date], ',', codesum['CodeSum2'][date])
        elif len(codesum['CodeSum1'][date]) > 1 and len(codesum['CodeSum2'][date]) == 1:
            stat2_null += 1
            aggregate.append(codesum['CodeSum1'][date])
            #print(date, codesum['CodeSum1'][date], ',', codesum['CodeSum2'][date])
        else:
            partial_matches += 1
            agg = codesum['CodeSum1'][date].split() + codesum['CodeSum2'][date].split()
            agg = pd.Series(list(set(agg)))
            agg = agg.str.cat(sep = ' ')
            aggregate.append(agg)
            #print(date, codesum['CodeSum1'][date], ',', codesum['CodeSum2'][date])            
            #print(agg)
            #print ('\n')    
        #print (no_matches)

print('\n',
      'exact_matches:', exact_matches, '\n',
      'both_null:', both_null, '\n',
      'partial_matches:', partial_matches, '\n',
      'stat1_null:', stat1_null, '\n',
      'stat2_null:', stat2_null, '\n',
      '\n',
      'total: ', exact_matches + both_null + stat1_null + stat2_null + partial_matches)

aggregate = pd.Series(aggregate)


 exact_matches: 232 
 both_null: 717 
 partial_matches: 348 
 stat1_null: 88 
 stat2_null: 87 
 
 total:  1472


In [17]:
#11b. checker
agg = codesum['CodeSum1']['2007-05-31'].split() + codesum['CodeSum2']['2007-05-31'].split()
print(codesum['CodeSum1']['2007-05-31'], ',', codesum['CodeSum2']['2007-05-31'])

agg = pd.Series(list(set(agg)))
agg = agg.str.cat(sep = ' ')
agg

RA HZ , RA BR


'HZ BR RA'

In [18]:
#12. update weather_new df
    #a.Repeat step #8 first: create weather_new df with index as Station
    #b. set unique dates
    #c. set calculated values

weather_new = weather.set_index('Station')
#weather_new.head()

In [19]:
#12b. set unique dates
weather_new = weather_new.loc[[1], :]
weather_new = weather_new.reset_index().drop(['Station'], axis = 1)
#weather_new.head()

In [20]:
#12c. set calculated values
for col in weather_new.drop(['Date', 'CodeSum'], axis = 1).columns:
    weather_new[col] = [newcols[col][i] for i in weather_new[col].index]

weather_new['CodeSum'] = [aggregate[i] for i in weather_new['CodeSum'].index]
#weather_new['CodeSum'][0:10]

In [21]:
#13. checking against original weather df
weather_new.head(10)
#weather.head(20)

,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,2007-05-01,83.5,51.0,67.5,14,51.0,56.5,0,2.5,0448,...,,0,0.0,0.0,0,29.14,29.82,2.20,26.0,9.4
1,2007-05-02,59.5,42.5,51.5,-3,42.0,47,13.5,0,0447,...,BR HZ,0,0.0,0.0,0,29.41,30.085,13.15,3.0,13.4
2,2007-05-03,66.5,47.0,57,2,40.0,49,8,0,0446,...,HZ,0,0.0,0.0,0,29.425,30.12,12.30,6.5,12.55
3,2007-05-04,72.0,50.0,58,4,41.5,50,7,0,0444,...,RA,0,0.0,0.0,0.00,29.335,30.045,10.25,7.5,10.6
4,2007-05-05,66.0,53.5,60,5,38.5,49.5,5,0,0443,...,,0,0.0,0.0,0,29.43,30.095,11.45,7.0,11.75
5,2007-05-06,68.0,50.5,59.5,4,30.0,46,5.5,0,0442,...,,0,0.0,0.0,0,29.595,30.285,14.10,10.5,14.75
6,2007-05-07,83.5,48.5,66,10,40.0,53.5,0,1,0441,...,RA,0,0.0,0.0,0.00,29.41,30.12,8.55,17.5,10.2
7,2007-05-08,81.0,57.0,69,12,57.5,62.5,0,4,0439,...,BR HZ,0,0.0,0.0,0.00,29.325,30.025,2.60,9.5,5.6
8,2007-05-09,76.5,62.0,69.5,13,59.5,63,0,4.5,0438,...,BR HZ,0,0.0,0.0,0.075,29.245,29.935,3.90,8.0,6.05
9,2007-05-10,83.5,57.5,70.5,14,52.0,60.5,0,5.5,0437,...,BR HZ,0,0.0,0.0,0,29.23,29.915,1.35,13.0,4


In [64]:
#14. Split weather_new df into train / test
    #train_set = years 2007, 09, 11, 13
    #test_set = years 2008, 10, 12, 14
    #spray = years 2011, 13
    #weather = years 2007-2014

#year in weather_new df
weather_new['Year'] = [int(weather_new['Date'][i][0:4]) for i in weather_new['Date'].index]

train_yrs = [2007, 2009, 2011, 2013]
test_yrs = [2008, 2010, 2012, 2014]

train_weather = weather_new.loc[weather_new['Year'].isin(train_yrs)]
test_weather = weather_new.loc[weather_new['Year'].isin(test_yrs)]

#print('train:', train_weather.shape, 'test:', test_weather.shape)
#print(weather_new[weather_new['Year'] == 2007].shape)
#train_weather.head(10)

In [65]:
test_weather.head(5)

,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,...,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Year
184,2008-05-01,73.5,48.0,61,8,51.0,56.5,4,0,0448,...,0,0.0,0.0,0.00,28.98,29.67,10.20,16.5,10.95,2008
185,2008-05-02,74.0,53.0,64,9,55.5,59,1,0,0447,...,0,0.0,0.0,1.01,28.82,29.525,12.85,19.5,14.05,2008
186,2008-05-03,56.0,43.5,50,-4,41.5,46,15,0,0446,...,0,0.0,0.0,0.01,29.045,29.69,10.80,25.0,12.7,2008
187,2008-05-04,65.5,38.5,52.5,-2,33.5,45,12.5,0,0444,...,0,0.0,0.0,0,29.345,30.02,5.35,23.5,5.95,2008
188,2008-05-05,74.5,50.0,62.5,8,38.0,50,2.5,0,0443,...,0,0.0,0.0,0,29.305,29.99,4.05,24.5,6.6,2008


In [66]:
#15. Handling merging training dfs on with different dates
    #len(list(pd.unique(test['Date']))) returns 95

#option 1: between last spray date and the date of spray, what is the average weather data?
    #for date == 2007-05-29, last 7 days worth of weather values --> prev_days = 7
    #for date == 2007-06-05, days between 2007-05-29 and 2007-06-05 worth of weather values --> prev_days = 7
    #for date == 2007-06-26, days between 2007-06-5 and 2007-06-26 worth of weather values --> prev_days = 21  
#caveat: interval of days is not be consistent

#option 2: straight 7 days prior worth of weather data
    #for any date, last 7 days worth of weather values --> prev days = 7

#option 3: month of date
    #for any date, average of entire month's worth of weather values

#option 4: day of date
    #for any date, weather values on matched date

In [41]:
#15. Option 4: combining datasets
    #spray df has no data past 2011/2013
combined = test.merge(test_weather, on = 'Date')
#combined.head()

In [67]:
#15. Option 2: for train_weather values between test['Date'] and test['Date'] - 7, average_values for test_weather[col]

#15-1. create new column in train_weather that is test['Date']
test_weather['Dateloc'] = pd.to_datetime(test['Date'])

    #convert date columns in train_weather to datetime type
test_weather['Date'] = pd.to_datetime(test_weather['Date'])

#15-2. create new column in train_weather that is train['Date'] - 7
from datetime import timedelta
test_weather['Prev7date'] = test_weather['Dateloc'] - timedelta(days = 7)

#15-3. create lookup table df
lookup = test_weather.iloc[:, :].drop(['Year', 'Dateloc', 'Prev7date'], axis = 1)
#lookup.set_index(['Date'], inplace = True)
lookup.head()

/Users/chelen/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/chelen/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/chelen/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://panda

,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
184,2008-05-01,73.5,48.0,61,8,51.0,56.5,4,0,0448,...,,0,0.0,0.0,0.00,28.98,29.67,10.20,16.5,10.95
185,2008-05-02,74.0,53.0,64,9,55.5,59,1,0,0447,...,TSRA RA BR,0,0.0,0.0,1.01,28.82,29.525,12.85,19.5,14.05
186,2008-05-03,56.0,43.5,50,-4,41.5,46,15,0,0446,...,BR RA,0,0.0,0.0,0.01,29.045,29.69,10.80,25.0,12.7
187,2008-05-04,65.5,38.5,52.5,-2,33.5,45,12.5,0,0444,...,,0,0.0,0.0,0,29.345,30.02,5.35,23.5,5.95
188,2008-05-05,74.5,50.0,62.5,8,38.0,50,2.5,0,0443,...,,0,0.0,0.0,0,29.305,29.99,4.05,24.5,6.6


In [68]:
test_weather.head()

,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,...,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Year,Dateloc,Prev7date
184,2008-05-01,73.5,48.0,61,8,51.0,56.5,4,0,0448,...,0.0,0.00,28.98,29.67,10.20,16.5,10.95,2008,2008-06-11,2008-06-04
185,2008-05-02,74.0,53.0,64,9,55.5,59,1,0,0447,...,0.0,1.01,28.82,29.525,12.85,19.5,14.05,2008,2008-06-11,2008-06-04
186,2008-05-03,56.0,43.5,50,-4,41.5,46,15,0,0446,...,0.0,0.01,29.045,29.69,10.80,25.0,12.7,2008,2008-06-11,2008-06-04
187,2008-05-04,65.5,38.5,52.5,-2,33.5,45,12.5,0,0444,...,0.0,0,29.345,30.02,5.35,23.5,5.95,2008,2008-06-11,2008-06-04
188,2008-05-05,74.5,50.0,62.5,8,38.0,50,2.5,0,0443,...,0.0,0,29.305,29.99,4.05,24.5,6.6,2008,2008-06-11,2008-06-04


In [69]:
lookup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 736 entries, 184 to 1471
Data columns (total 21 columns):
Date           736 non-null datetime64[ns]
Tmax           736 non-null float64
Tmin           736 non-null float64
Tavg           736 non-null object
Depart         736 non-null object
DewPoint       736 non-null float64
WetBulb        736 non-null object
Heat           736 non-null object
Cool           736 non-null object
Sunrise        736 non-null object
Sunset         736 non-null object
CodeSum        736 non-null object
Depth          736 non-null object
Water1         736 non-null float64
SnowFall       736 non-null object
PrecipTotal    736 non-null object
StnPressure    736 non-null object
SeaLevel       736 non-null object
ResultSpeed    736 non-null float64
ResultDir      736 non-null float64
AvgSpeed       736 non-null object
dtypes: datetime64[ns](1), float64(6), object(14)
memory usage: 126.5+ KB


In [86]:
#15-4. update test_weather df: drop unnecessary rows
    #keep unique dates from test set (95 unique values)

test_weather = test_weather.reset_index()
test_weather = test_weather.drop(['index'], axis = 1)

test_weather = test_weather.loc[:94, :]

test_weather['Dateloc'] = list(test['Date'].unique())
test_weather['Prev7date'] = pd.to_datetime(test_weather['Dateloc']) - timedelta(days = 7)
test_weather = test_weather.drop(['Date'], axis = 1)

#test_weather.head()

,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,...,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Year,Dateloc,Prev7date
0,73.5,48.0,61,8,51.0,56.5,4,0,0448,1849,...,0.0,0.00,28.98,29.67,10.20,16.5,10.95,2008,2008-06-11,2008-06-04
1,74.0,53.0,64,9,55.5,59,1,0,0447,1850,...,0.0,1.01,28.82,29.525,12.85,19.5,14.05,2008,2008-06-17,2008-06-10
2,56.0,43.5,50,-4,41.5,46,15,0,0446,1851,...,0.0,0.01,29.045,29.69,10.80,25.0,12.7,2008,2008-06-24,2008-06-17
3,65.5,38.5,52.5,-2,33.5,45,12.5,0,0444,1852,...,0.0,0,29.345,30.02,5.35,23.5,5.95,2008,2008-07-01,2008-06-24
4,74.5,50.0,62.5,8,38.0,50,2.5,0,0443,1853,...,0.0,0,29.305,29.99,4.05,24.5,6.6,2008,2008-07-04,2008-06-27


In [87]:
#15-5a. numerical cols: calculate prev7day_average values by unique test['Date']

allcols = {}

for col in test_weather.drop(['CodeSum', 'Year', 'Dateloc', 'Prev7date'], axis = 1).columns:
    #print(col)
    colvals = []
    
    for date in test_weather['Dateloc']:
        vals = []
        t = 7
    
        while t > 0:
            lookup_date = pd.to_datetime(date) - timedelta(days = t)
            val = lookup[lookup['Date'] == lookup_date][col].values[0]
            vals.append(val)
            #print(t, lookup_date, val)
            t -= 1
        aveval = np.average(pd.to_numeric(vals)).round(1)
        #print(vals)
        #print('dateloc:', date, aveval, '\n')
        
        colvals.append(aveval)
    
    #print(colvals)
    allcols[col] = colvals

print(allcols.keys())

dict_keys(['Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint', 'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset', 'Depth', 'Water1', 'SnowFall', 'PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed'])


In [88]:
#15-5b. text cols: aggregate all values by unique test['Date']
    #col CodeSum

for col in test_weather.loc[:, ['CodeSum']].columns:
    #print(col)
    colvals = {}
    
    for date in test_weather['Dateloc']:
        #print(date)
        vals = []
        t = 7
    
        while t > 0:
            lookup_date = pd.to_datetime(date) - timedelta(days = t)
            val = lookup[lookup['Date'] == lookup_date][col].values[0].split()
            vals.append(val)
            #print(t, lookup_date, val)
            t -= 1

        #flatten list of lists into one list + return unique CodeSum values
        vals = [item for sublist in vals for item in sublist]
        vals = list(set(vals))
        #print(date, vals)
        
        colvals[date] = vals
        #print('\n')
    
    #print(colvals.items())


In [92]:
#15-5. checker
#print(allcols.items())
#allcols['Tmax'][0:10]
#print(colvals.items())

In [93]:
#15-6. input prev7day_ave values into train_weather by col
    #a. numeric cols
    #b. text col
for col in test_weather.drop(['Year', 'Dateloc', 'Prev7date', 'CodeSum'], axis = 1).columns:
    test_weather[col] = [allcols[col][i] for i in test_weather[col].index]

codesums = [colvals[date] for date in test_weather['Dateloc']]
test_weather['CodeSum'] = [codesums[row] for row in range(len(test_weather['CodeSum']))]

#test_weather.head()
#test_weather.loc[:, ['Dateloc', 'CodeSum']].head()

,Dateloc,CodeSum
0,2008-06-11,"[FG+, RA, SQ, BR, DZ, HZ, TS, TSRA, FG]"
1,2008-06-17,"[BR, TSRA, RA, TS]"
2,2008-06-24,"[TS, RA, TSRA]"
3,2008-07-01,"[BR, HZ, RA, TSRA]"
4,2008-07-04,"[RA, BR, HZ, TS, TSRA]"


In [95]:
#16. clean train_weather: drop unnecessary cols, rename Dateloc to Date
test_weather = test_weather.drop(['Year', 'Prev7date'], axis = 1)
test_weather.rename(columns={'Dateloc': 'Date'}, inplace=True)
#test_weather.head()

,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,...,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Date
0,82.3,64.7,73.7,7.3,65.2,68.2,0.0,8.7,416.9,1924.0,...,0.0,0.0,0.0,0.3,29.1,29.8,9.9,18.5,11.7,2008-06-11
1,82.4,63.1,72.9,4.7,56.9,63.4,0.0,7.9,416.0,1927.3,...,0.0,0.0,0.0,0.1,29.2,29.9,10.1,23.2,10.9,2008-06-17
2,79.4,57.4,68.8,-1.4,50.4,58.1,0.0,3.8,416.3,1930.0,...,0.0,0.0,0.0,0.0,29.3,29.9,5.1,20.1,7.1,2008-06-24
3,81.7,63.6,72.9,1.3,59.0,64.3,0.0,7.9,418.1,1931.0,...,0.0,0.0,0.0,0.1,29.2,29.9,7.5,25.3,9.1,2008-07-01
4,81.0,61.4,71.4,-0.7,56.1,62.2,0.1,6.5,419.4,1931.0,...,0.0,0.0,0.0,0.1,29.2,29.8,8.9,23.1,10.7,2008-07-04


In [96]:
#17. merge train loc + weather dfs together
    #spray has different dates
test_locwea = test.merge(test_weather, on = 'Date')

#test_locwea.head()
#test_locwea.columns

In [100]:
#18. save test_weather to_csv
test_weather.to_csv('test_weather')

In [1]:
#making some changes to test git fetch